# Validation of the mass source term implementation for a WC problem

In [ ]:
from trustutils import run

run.introduction("Elie Saikali & Adrien Bruneton")
run.TRUST_parameters("1.9.0")

## Description

This jupyter notebook aims at validating the implementation of the mass loss source term for a WC multi-species problem. Both VDF and VEF discretizations are considered where for each we consider two simulations: either for two or three species ! 

In what follows we recall at first the classical WC governing equation and how the system is modified to take into account the mass loss from the system. Afterwards, we recall the pressure-projection algorithm used with the WC problem (thats for you Benoit) ! Test cases and results are finally presented.

In [ ]:
import os, sys

run.reset()
run.initBuildDirectory()
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)

## Multi-species WC Governing equations


In general, the dimensional WC system of governing equations reads as

\begin{align}
&\frac{\partial {\rho}}{\partial t}+\frac{\partial }{\partial x_i} \bigg({\rho} {u}_i\bigg) = 0, \label{mass}\\
&\frac{\partial {\rho}{ u}_j }{\partial t}+ \frac{\partial }{\partial x_i} \bigg({\rho} {u}_j {u}_i \bigg) =-\frac{\partial{ P}}{\partial x_j} + \frac{\partial{\tau}_{ij}}{\partial x_i}+{\rho} g_j, \label{mom}\\
&\rho c_p\frac{\partial T}{\partial t}+  \rho c_p {u}_i\frac{\partial T}{\partial x_i}=\frac{\partial }{\partial x_i}\bigg( {\rho} \lambda \frac{\partial T}{\partial x_i}\bigg)+\frac{dP_{tot}}{dt},\\
&  \rho\frac{\partial{ Y}_i}{\partial t}+ \rho {u}_i\frac{\partial Y_i}{\partial x_i}=\frac{\partial }{\partial x_i}\bigg( {\rho} \  D \frac{\partial {Y }_i}{\partial x_i}\bigg),\label{spec}\\
&{\rho}={M}_{mix}\frac{P_{tot}}{RT}, \label{state}
\end{align}

where ${\rho}$ is the mixture's density and ${u}_i$ the mass average component of the velocity vector ${\textbf{u}}=({u}_1,{u}_2,{u}_3)$ and $g_j=(0,0,-g)$ the gravity vector. ${Y}_i$ is the mass fraction of species $i$ satisfying $\sum {Y}_i=1$. 

$P_{tot}$ is the total pressure; ie: the sum of the thermo-dynamic pressure $p=1$ bar (in general) and the hydro-dynamic pressure $P$ that is an unknown of the NS equation. Recall here that this is the basic difference between the QC and WC formulations. Here, the pressure of the state equation can vary in space too !

$D$ corresponds to the mixture diffusion coefficient of both species,  $R= 8.314$ J.K$^{-1}$.mol$^{-1}$ the specific gas constant and $M_{mix}=\displaystyle\left(\sum_{i=1}^2  \frac{{Y}_i}{M_i}\right)^{-1}$ the mixing molar mass.

 ${\tau}_{ij}= 2\mu{e}_{ij}$ is the viscous stress tensor for Newtonian fluids with
$${e}_{ij}= \frac{1}{2}\left(\frac{\partial  u_i}{\partial x_j} + \frac{\partial u_j}{\partial x_i}\right) -\frac{1}{3}\delta_{ij}\frac{\partial u_k}{\partial x_k},$$
and $\delta_{ij}$ the Kronecker symbol.  $\mu$ denotes the mixture's dynamic viscosity calculated as a function of the mass fractions and fluids physical properties using the Wilke's formulation. 

For example, for the species the formula reads as follows
\begin{equation}\label{wilke}
        \mu = \frac{Y_1\mu_1}{Y_1\phi_{11}+Y_2\phi_{12}}+\frac{Y_2\mu_2}{Y_1\phi_{21}+Y_2\phi_{22}},
\end{equation}
        where $\phi_{ij}$ is a set of dimensionless constants calculated as
\begin{equation}
        \phi_{ij}= \displaystyle\frac{\displaystyle\frac{\textrm{M}_i}{\textrm{M}_j}\left[ 1+\left(\displaystyle\frac{\mu_i}{\mu_j} \right)^{1/2} \left( \displaystyle\frac{\textrm{M}_j}{\textrm{M}_i} \right)^{1/4}\right]^2}{ \left[ 8\left(1+\displaystyle\frac{\textrm{M}_i}{\textrm{M}_j} \right) \right]^{1/2}}\quad : \quad i,j=\{1,2\}.
\end{equation}


### Mass loss

The mass conservation equation in its original form insures the mass balance in the system. Recall that this equation is not solved (explicitly) in the code but used in the projection algorithm on the RHS of the elliptic pressure Poisson's equation. This means that if a mass flux is prescribed at a considered boundary for a single species equation, the entire mixture is affected and not only the desired species.

In order to control that (consumption of H2 only along a boundary in a PEMFC for example), a mass-loss source term should be considered. For that, the iso-thermal system of GE becomes (temperature equation not considered)

\begin{align}
&\frac{\partial {\rho}}{\partial t}+\frac{\partial }{\partial x_i} \bigg({\rho} {u}_i\bigg) = -\bf{S},\\
&\frac{\partial {\rho}{ u}_j }{\partial t}+ \frac{\partial }{\partial x_i} \bigg({\rho} {u}_j {u}_i \bigg) =-\frac{\partial{ P}}{\partial x_j} + \frac{\partial{\tau}_{ij}}{\partial x_i}+{\rho} g_j,\\
&  \rho\frac{\partial{ Y}_i}{\partial t}+ \rho {u}_i\frac{\partial Y_i}{\partial x_i}=\frac{\partial }{\partial x_i}\bigg( {\rho} \  D \frac{\partial {Y }_i}{\partial x_i}\bigg) + \bf{Y_i S},\\
&{\rho}={M}_{mix}\frac{P_{tot}}{RT}, 
\end{align}

where $\textbf{S}$ [Kg.m$^{-3}$.s$^{-1}$] is a volumic source term applied on the first cell in contact with the boundary where the mass flux $\mathcal{F}$ [Kg.m$^{-2}$.s$^{-1}$] is prescribed on the species equation. In particular, $$\bf{S} = \mathcal{F} \times \mathcal{S} / \mathcal{V},$$
where $\mathcal{S}$ and $\mathcal{V}$ are respectively the surface area and the voulme of the boundary element. Note that as far as we consider a single mixture velocity $u_i$, $\mathcal{F}$ is considered as a pure diffusive flux (no convective contribution).

We emphasize that the presented formulation is generic and works for cases where different mass fluxes are imposed for different species. For example, say we impose $\mathcal{F}_1$ for species equation 1 and $\mathcal{F}_2$ for species equation 2, the source term is considered as $$\bf{S}=\bf{S}_1+\bf{S}_2,$$ where $\bf{S}_1 = \mathcal{F}_1 \times \mathcal{S} / \mathcal{V}$ and $\bf{S}_2 = \mathcal{F}_2 \times \mathcal{S} /  \mathcal{V}$.

### TRUST implementation

The new formulation is currently available for both VDF and VEF discretizations. In accordance with the previous system of GE, the code is modified basically in two places; for the projection algorithm to take into account $\textbf{S}$ in the RHS of the Poisson equation, and for the species equations to add the contribution ! In what follows, we recall the incremental projection algorithm (Chorin) employed in the WC model.


#### Chorin's algorithm 

Consider the WC NS equation 

$$\frac{\partial {\rho}{ u}_j }{\partial t}+ \frac{\partial }{\partial x_i} \bigg({\rho} {u}_j {u}_i \bigg) =-\frac{\partial{ P}}{\partial x_j} + \frac{\partial{\tau}_{ij}}{\partial x_i}+{\rho} g_j$$

At the prediction step we have

$$\frac{ ({\rho}{ u}_j)^{\ast} - ({\rho}{ u}_j)^{n} }{d t}+ \frac{\partial }{\partial x_i} \bigg({\rho} {u}_j {u}_i \bigg)\bigg|^{n+1} =-\frac{\partial{ P}}{\partial x_j}\bigg|^{n} + \frac{\partial{\tau}_{ij}}{\partial x_i}\bigg|^{n+1}+{\rho} g_j\bigg|^{n+1}$$

At the correction we have

$$\frac{ ({\rho}{ u}_j)^{n+1} - ({\rho}{ u}_j)^{n} }{d t}+ \frac{\partial }{\partial x_i} \bigg({\rho} {u}_j {u}_i \bigg)\bigg|^{n+1} =-\frac{\partial{ P}}{\partial x_j}\bigg|^{n+1} + \frac{\partial{\tau}_{ij}}{\partial x_i}\bigg|^{n+1}+{\rho} g_j\bigg|^{n+1}$$

Taking the difference we get 

$$ \frac{ ({\rho}{ u}_j)^{\ast} - ({\rho}{ u}_j)^{n+1} }{d t} = \frac{\partial{ \Pi}}{\partial x_j},$$

where $\Pi=P^{n+1}-P^{n}$ is the pressure increment. Finally, the Poisson equation is obtained by applying the divergence operator on both sides of the equation

$$ \frac{ \frac{\partial}{\partial x_j}({\rho}{ u}_j)^{\ast} - \frac{\partial}{\partial x_j} ({\rho}{ u}_j)^{n+1} }{d t} =\frac{\partial}{\partial x_j} \frac{\partial{ \Pi}}{\partial x_j}.$$

Using the mass equation 
$$\frac{\partial {\rho}}{\partial t}+\frac{\partial }{\partial x_i} \bigg({\rho} {u}_i\bigg) = -\bf{S}, $$

the pressure Poisson equation becomes

$$ \displaystyle\frac{ \frac{\partial}{\partial x_j}({\rho}{ u}_j)^{\ast} +\frac{\partial {\rho}}{\partial t}\bigg|^{n+1}+\bf{S}\bigg|^{n+1} }{d t} =\frac{\partial}{\partial x_j} \frac{\partial{ \Pi}}{\partial x_j}.$$

After solving this equation, the pressure and velocity fields at time $n+1$ are obtained as

$$P^{n+1}=\Pi + P^n$$
$$u_j^{n+1} = \frac{1}{\rho^{n+1}}({\rho}{ u}_j)^{n+1}$$

## Computation

We consider three gases O$_2$, N$_2$ and H$_2$O (M1 = 32 e$^{-3}$, M2 = 28 e$^{-3}$ Kg.mol$^{-1}$ and M3 = 18 e$^{-3}$ Kg.mol$^{-1}$) injected in a channel (1x50x1 e$^{-3}$ m for VDF and 1x30x1 e$^{-3}$ m for VEF). See the meshes in the next figures. 

At the injection we fix $Y_1=0.5$ ($Y_2=0.2$ for multi-species case) and $\rho u=0.63$ Kg.m$^{-2}$.s$^{-1}$.

At the outlet, the hydrostatic pressure is fixed. On the wall boundaries (not in contact with the AME), a no-slip BC is used.

At the AME boundary, a no-slip BC is for NS and a mass flux is imposed for the conservation equation of $Y_1$ (and $Y_2$ for multi-species case). The flux is calculated in order to consume along this boundary $\textbf{all the injected}$ O$_2$ and H$_2$O.

In [ ]:
class ValueSet:
    def __init__(self):
        self.rho_u = {}
        self.rhoYO2_u = {}
        self.rhoYH2O_u = {}
        self.rhoYN2_u = {}


def readout(path, name_file):
    data=[]
    file = os.path.join(path,name_file)
    with open(file) as f:
        for l in f:
            if l.startswith('#'):
                header=l[1:].split()
            elif len(l.split())>1:
                d={}
                for i,j in zip(header,l.split()):
                    d[i]=float(j)                    
    return d

def compute_surfaces(discretization):
    if discretization == "VDF":
        # num of nodes in mesh VDF
        nx, ny, nz = 6, 51, 6
        lx, ly, lz = 1.e-3, 50.e-3, 1.e-3
    elif discretization == "VEF":
        nx, ny, nz = 3, 13, 3
        lx, ly, lz = 1.e-3, 30.e-3, 1.e-3
            
    S_inj = lx*lz # section entree
    s_ame = lx*(ly-10e-3) # in m2
        
    return S_inj, s_ame

P = 1e5  # pression gaz (Pa)
T = 375.  # Temperature (K)
R = 8.314472

MO2 = 32.e-3 # O2 in kg/mol
MN2 = 28.e-3 # N2 in kg/mol
MH2O = 18.e-3 # H2O in kg/mol

class CommonData:

    def __init__(self, Y_O2, Y_H2O, ch):

        self.ch = ch
        self.Y_O2 = Y_O2
        self.stoechio = 1.5
        
        if self.ch=="binaire":
            self.Y_H2O = 0.
        elif self.ch=="multi":
            self.Y_H2O = Y_H2O
        
        self.Y_N2 = 1.-self.Y_O2-self.Y_H2O
        self.rho_mix_init = 1./(R*T/P*(self.Y_O2/MO2+self.Y_N2/MN2+self.Y_H2O/MH2O))
        

    def compute_fluxes(self, rhou, discretization): 
        
        S_inj, s_ame = compute_surfaces(discretization)
        
        self.v_inj = rhou/self.rho_mix_init
    
        self.debit_Y_O2_inj = self.v_inj * self.rho_mix_init * S_inj * self.Y_O2 # debit mass O2 en entree [kg/s]
        flux_Y_O2_ame = -self.debit_Y_O2_inj / self.stoechio / s_ame # ./m2/s  
    
        if self.ch=="multi":
            self.debit_Y_H2O_inj = self.v_inj * self.rho_mix_init * S_inj * self.Y_H2O # debit mass H2O en entree [kg/s]
        else:
            self.debit_Y_H2O_inj = 0.
        
        flux_Y_H2O_ame = -self.debit_Y_H2O_inj / self.stoechio / s_ame # ./m2/s
        
        return flux_Y_O2_ame, flux_Y_H2O_ame

    def check_fluxes(self, path, discretization):
        
        S_inj, s_ame = compute_surfaces(discretization)

        trust = ValueSet()
        
        boundaries=["in","ame","out"]
        
        root_name = 'flux_'+discretization+'_'+self.ch
        
        trust.rho_u=readout(path, root_name+'_pb_Debit.out')
        
        # Convention: rho_u>0 => flux entrant
        for i in boundaries:
            trust.rho_u[i] *= -1


        if self.ch=="binaire":
            conv1=readout(path, root_name+'_pb_Convection_Espece_Binaire.out')
            diff1=readout(path, root_name+'_pb_Diffusion_Espece_Binaire.out')
        elif self.ch=="multi":
            conv1=readout(path, root_name+'_pb_Convection_FRACTION_MASSIQUE0.out')
            diff1=readout(path, root_name+'_pb_Diffusion_FRACTION_MASSIQUE0.out')
            conv2=readout(path, root_name+'_pb_Convection_FRACTION_MASSIQUE1.out')
            diff2=readout(path, root_name+'_pb_Diffusion_FRACTION_MASSIQUE1.out')

        for i in boundaries:
            trust.rhoYO2_u[i] = conv1[i] + diff1[i]
            if self.ch=="multi":
                trust.rhoYH2O_u[i] = conv2[i] + diff2[i]
            else:
                trust.rhoYH2O_u[i] = 0.

        source_term = -trust.rhoYO2_u["in"]/self.stoechio - trust.rhoYH2O_u["in"]/self.stoechio        
        trust.rho_u["ame"] += source_term

        for i in boundaries:
            trust.rhoYN2_u[i] = trust.rho_u[i] - trust.rhoYO2_u[i] - trust.rhoYH2O_u[i]

        # calcul des valeurs theoriques
        theo=ValueSet()
        theo.rho_u["in"] = self.rho_mix_init*self.v_inj*S_inj
        theo.rho_u["ame"] = -self.debit_Y_O2_inj/self.stoechio-self.debit_Y_H2O_inj/self.stoechio
        theo.rhoYO2_u["in"] = self.debit_Y_O2_inj
        theo.rhoYO2_u["ame"] = -self.debit_Y_O2_inj/self.stoechio
        theo.rhoYH2O_u["in"] = self.debit_Y_H2O_inj
        theo.rhoYH2O_u["ame"] = -self.debit_Y_H2O_inj/self.stoechio
        for i in [theo.rho_u,theo.rhoYO2_u,theo.rhoYH2O_u]:
            i["out"] = -i["in"]-i["ame"]

        for i in boundaries:
            theo.rhoYN2_u[i] = theo.rho_u[i] - theo.rhoYO2_u[i] - theo.rhoYH2O_u[i]

        print ("=================================================================")
        for i in boundaries:
            i_affich = i
            if i == "ame":
                i_affich = "ame + SOURCES"
            print("boundary: "+ i_affich)
            for j in ["rho_u","rhoYO2_u", "rhoYH2O_u", "rhoYN2_u"]:
               a=getattr(trust,j)[i]*1e9
               b=getattr(theo,j)[i]*1e9
               print("%4s %6s  trust: %12.5f   theo: %12.5f [1E-9kg/s]"%(i,j,a,b))
            a = (getattr(trust,"rho_u")[i] - getattr(trust,"rhoYO2_u")[i] - getattr(trust,"rhoYH2O_u")[i] - getattr(trust,"rhoYN2_u")[i]) *1e9
            b = (getattr(theo,"rho_u")[i] - getattr(theo,"rhoYO2_u")[i] - getattr(theo,"rhoYH2O_u")[i] - getattr(theo,"rhoYN2_u")[i]) *1e9
            print("")
            print("%4s TOTAL  trust: %12.5f   theo: %12.5f [1E-9kg/s]"%(i,a,b))
            print ("=================================================================")

        print("")
        print ("SUM OVER ALL BOUNDARIES PER VARIABLE")

        print ("====================================")

        for j in ["rho_u","rhoYO2_u", "rhoYH2O_u", "rhoYN2_u"]:
            v_theo = 0.0
            v_real = 0.0
            for i in boundaries:
               a=getattr(trust,j)[i]*1e9
               b=getattr(theo,j)[i]*1e9
               v_real += a
               v_theo += b
            print("Var ", j, " trust: %12.5f  theo: %12.5f" % (v_real, v_theo) )

        

        

In [ ]:
Y_O2 = 0.5
Y_H2O = 0.2 # ignored for the multi-species case

rhou = 0.63

mixture = {}

for dis in ["VDF", "VEF"]:
    for ch in ["binaire", "multi"]:
        mixture[f"{ch}"] = CommonData(Y_O2, Y_H2O, f"{ch}")
        flux_Y_O2_ame, flux_Y_H2O_ame = mixture[f"{ch}"].compute_fluxes(rhou, f"{dis}")
        
        os.system(f"mkdir -p {dis}/{ch}")
        run.addCaseFromTemplate(f"flux_{dis}_{ch}.data",targetDirectory=f"{dis}/{ch}",dic={"flux1_": str(flux_Y_O2_ame), "flux2_": str(flux_Y_H2O_ame)})

run.printCases()
run.runCases()

## Binary mixture test cases

### VDF

In [ ]:
from trustutils import visit

a=visit.Show("./VDF/binaire/flux_VDF_binaire.lata","Pseudocolor","FRACTION_MASSIQUE_ELEM_dom")
a.normal3D([0.707549, -0.414716, 0.572175])
a.plot()

In [ ]:
mixture["binaire"].check_fluxes(f'{run.BUILD_DIRECTORY}/VDF/binaire/', "VDF")

As you can note, only species $Y_1$ is consumed on the AME boundary !

### VEF

In [ ]:
a=visit.Show("./VEF/binaire/flux_VEF_binaire.lata","Pseudocolor","FRACTION_MASSIQUE_ELEM_dom")
a.normal3D([0.707549, -0.414716, 0.572175])
a.plot()

In [ ]:
mixture["binaire"].check_fluxes(f'{run.BUILD_DIRECTORY}/VEF/binaire/', "VEF")

As you can note, only species $Y_1$ is consumed on the AME boundary !

## Multi-species mixture test cases

### VDF

In [ ]:
a=visit.Show("./VDF/multi/flux_VDF_multi.lata","Pseudocolor","FRACTION_MASSIQUE0_ELEM_dom")
a.normal3D([0.707549, -0.414716, 0.572175])
a.plot()

In [ ]:
mixture["multi"].check_fluxes(f'{run.BUILD_DIRECTORY}/VDF/multi/',"VDF")

As you can note, only species $Y_1$ and $Y_2$ are consumed on the AME boundary ! The ratio is 2.5, which is equal to the ratio at the injection !! Thats perfect ...

### VEF

In [ ]:
a=visit.Show("./VEF/multi/flux_VEF_multi.lata","Pseudocolor","FRACTION_MASSIQUE0_ELEM_dom")
a.normal3D([0.707549, -0.414716, 0.572175])
a.plot()

In [ ]:
mixture["multi"].check_fluxes(f'{run.BUILD_DIRECTORY}/VEF/multi/',"VEF")

As you can note, only species $Y_1$ and $Y_2$ are consumed on the AME boundary ! The ratio is 2.5, which is equal to the ratio at the injection !! Thats perfect ...

## Conclusions and advices for users

We conclude that the mass source term is coded correctly for both VDF and VEF discretizations. It works with explicit, semi-implicit and fully implicit time schemes.

### Advices

- For VEF, use EF_Stab scheme for convection !

- Make sure that the time scheme is stable ! large time steps can lead to bad results ... Attention for the facsec  (specially when using a fully-implicit scheme) !